In [1]:
from bayes_sensor import *
from config import VALID_CONFIG
from pprint import pprint

I wish to understand how the bayesian_sensor operates, lets do an investigation

### Code refs
* https://github.com/home-assistant/home-assistant/blob/dev/homeassistant/components/binary_sensor/bayesian.py code
* https://home-assistant.io/components/binary_sensor.bayesian/ docs
* https://github.com/jlmcgehee21/smart_hass#binary-bayes-introspection HA sensor author script for working with bayes sensor  
* https://github.com/home-assistant/home-assistant/tree/a1f238816b6130aee2ac88fe9da54ba8f65225f3 Very early home-assistant commit to better understand HA architechture


### Bayes
* https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/02-Discrete-Bayes.ipynb Recommended reading
* https://en.wikipedia.org/wiki/Bayes%27_theorem wikipedia on Bayes

$$\mbox{probability}= \frac{\mbox{likelihood} \times \mbox{prior}}{\mbox{normalization}}$$

A bayesian_sensor is ON if:

$$\mbox{probability} > \mbox{threshold}$$ 

We assign a probability to each observation indicating how likely it is to indicate the sensor is on, this is prob_given_true. If we assume a perfect observation, then prob_given_false = 1 - prob_given_true such that prob_given_false + prob_given_true = 1. However real observations may be noisy, in which case prob_given_false + prob_given_true != 1. Therefore the bayesian sensor allows us to set both prob_given_true and prob_given_false.

The function update_probability() is used to calculate the (posterior) probability.

In [2]:
def update_probability(prior, prob_true, prob_false):
    """Update probability using Bayes' rule."""
    numerator = prob_true * prior
    denominator = numerator + prob_false * (1 - prior)

    probability = numerator / denominator
    return probability

DEFAULT_PROBABILITY_THRESHOLD = 0.5

A valid config from the docs 

Lets calculate the (posterior) probability for a bayesian sensor with a prior = 0.5 given a single sensor reading that has prob_given_true = 0.5.

In [3]:
prior = 0.3
prob_given_true = 0.5
prob_given_false = 1 - prob_given_true
probability = update_probability(prior, prob_given_true, prob_given_false)
probability = round(probability, 2)
state = 'ON' if probability > DEFAULT_PROBABILITY_THRESHOLD else 'OFF'
print("Given a prior {} and prob_given_true of {} for an accurate sensor the (posterior) probability is {} and the sensor is {}".format(prior, prob_given_true, probability, state))

Given a prior 0.3 and prob_given_true of 0.5 for an accurate sensor the (posterior) probability is 0.3 and the sensor is OFF


Since this sensor only has a 50:50 chance of indicating the true state of the bayesian sensor, our prior == (posterior) probability and we have no reason to change our belief in the state of the bayesian_sensor.

Now lets say this sensor is a pretty good indicator such that prob_given_true = 0.75.

In [4]:
prob_given_true = 0.75
prob_given_false = 1 - prob_given_true
probability = update_probability(prior, prob_given_true, prob_given_false)
probability = round(probability, 2)
state = 'ON' if probability > DEFAULT_PROBABILITY_THRESHOLD else 'OFF'
print("Given a prior {} and prob_given_true of {} for an accurate sensor the (posterior) probability is {} and the sensor is {}".format(prior, prob_given_true, probability, state))

Given a prior 0.3 and prob_given_true of 0.75 for an accurate sensor the (posterior) probability is 0.56 and the sensor is ON


Hmm more study required..

## Bayesian sensor
Lets now import the actual sensor used in HA.

In [6]:
VALID_CONFIG

{'device_class': 'binary_device',
 'name': 'in_bed',
 'observations': [{'entity_id': 'sensor.bedroom_motion',
   'platform': 'state',
   'prob_given_true': 0.5,
   'to_state': 'on'},
  {'entity_id': 'sun.sun',
   'platform': 'state',
   'prob_given_true': 0.7,
   'to_state': 'below_horizon'}],
 'prior': 0.25,
 'probability_threshold': 0.95}

Create a bayesian sensor to explore properties

In [7]:
b_sensor = setup_platform(VALID_CONFIG)

updates are handeled through async_threshold_sensor_state_listener()

In [8]:
pprint(vars(b_sensor))

{'_deviation': False,
 '_device_class': 'binary_device',
 '_name': 'in_bed',
 '_observations': [{'entity_id': 'sensor.bedroom_motion',
                    'id': 0,
                    'platform': 'state',
                    'prob_given_true': 0.5,
                    'to_state': 'on'},
                   {'entity_id': 'sun.sun',
                    'id': 1,
                    'platform': 'state',
                    'prob_given_true': 0.7,
                    'to_state': 'below_horizon'}],
 '_probability_threshold': 0.95,
 'current_obs': OrderedDict(),
 'entity_obs': {'sensor.bedroom_motion': [{'entity_id': 'sensor.bedroom_motion',
                                           'id': 0,
                                           'platform': 'state',
                                           'prob_given_true': 0.5,
                                           'to_state': 'on'},
                                          {'entity_id': 'sun.sun',
                                           'id

In [9]:
b_sensor.device_state_attributes

{'observations': [], 'probability': 0.25, 'probability_threshold': 0.95}

In [10]:
b_sensor.state

'off'

In [11]:
b_sensor.entity_obs.keys()

dict_keys(['sensor.bedroom_motion', 'sun.sun'])

In [12]:
b_sensor.current_obs.values()

odict_values([])